***GENERATED CODE FOR RegressionAppforPredictingForecastingMilkProduction PIPELINE***

**CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM**

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(inStages, inStagesData, stageId, spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(inStages, inStagesData, stageId, spark, config):
        return inStagesData.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                                        delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


**TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA**

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_transform"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    indexed = indexed.drop(feature).withColumnRenamed(outcol, feature)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        feature).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            feature, dfReturn[feature].cast(IntegerType()))
        return changed_type_df
    # changed_type_df.show(3)
    return dfReturn


Feature_Transformations_Methods = {
    "String Indexer": StringIndexerTransform,

}


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, spark, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        for transformation in featureData:
            feature = transformation["feature"]
            if transformation["transformation"] != '' and transformation["selected"].lower() == "true" and not (feature.__contains__("_transform")):
                transformationDF = Feature_Transformations_Methods["%s" % transformation["transformation"]](
                    transformationDF, transformation)

        display(transformationDF.limit(2).toPandas())
        return transformationDF


**AUTOML FUNCTIONS**

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    df.columns.intersection(listOfFeatures)
    X = df.drop(label, axis=1).values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=15,
                              n_jobs=-1, random_state=25, population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))


**READING DATAFRAME**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

try: 
	RegressionAppforPredictingForecastingMilkProduction_DBFS = DBFSConnector.fetch([], {}, "5eac6f0f86c504fa4f86319a", spark, "{'url': '/Demo/MilkProductionTrain.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")

except Exception as ex: 
	logging.error(ex)


**TRANSFORMING DATAFRAME**

In [ ]:
try: 
	RegressionAppforPredictingForecastingMilkProduction_AutoFE = TransformationMain.run(RegressionAppforPredictingForecastingMilkProduction_DBFS, spark,json.dumps( {"FE": [{"transformationsData": {}, "feature": "Year_Month", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "87", "mean": "2004.14", "stddev": "5.58", "min": "1995.07", "max": "2013.12", "missing": "0"}, "transformation": ""}, {"transformationsData": {}, "feature": "Month_Number", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "87", "mean": "6.45", "stddev": "3.66", "min": "1", "max": "12", "missing": "0"}}, {"transformationsData": {}, "feature": "Year", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "87", "mean": "2004.08", "stddev": "5.58", "min": "1995", "max": "2013", "missing": "0"}}, {"transformationsData": {"feature_label": "Month"}, "feature": "Month", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "87", "mean": "", "stddev": "", "min": "Apr", "max": "Sept", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {}, "feature": "Cotagecheese_Prod", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "87", "mean": "2.81", "stddev": "0.5", "min": "1.941", "max": "4.377", "missing": "0"}, "transformation": ""}, {"transformationsData": {}, "feature": "Icecream_Prod", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "87", "mean": "70.53", "stddev": "11.21", "min": "50.85", "max": "92.091", "missing": "0"}, "transformation": ""}, {"transformationsData": {}, "feature": "Milk_Prod", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "87", "mean": "2.95", "stddev": "0.47", "min": "2.03", "max": "3.744", "missing": "0"}, "transformation": ""}, {"transformationsData": {}, "feature": "N_CA_Fat_Price", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "87", "mean": "1.57", "stddev": "0.39", "min": "0.8934", "max": "2.5285", "missing": "0"}, "transformation": ""}, {"feature": "Month_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "87", "mean": "4.6", "stddev": "3.42", "min": "0.0", "max": "13.0", "missing": "0"}}]}))

except Exception as ex: 
	logging.error(ex)


**TRAIN MODEL**

In [ ]:
try: 
	functionRegression(RegressionAppforPredictingForecastingMilkProduction_AutoFE, ["Year_Month", "Month_Number", "Year", "Month", "Cotagecheese_Prod", "Icecream_Prod", "Milk_Prod"], "N_CA_Fat_Price")

except Exception as ex: 
	logging.error(ex)
